In [99]:
# pip install google-generativeai
import os
import gradio as gr
from dotenv import load_dotenv
import google.generativeai as palm
import re
import requests
import json
from textwrap import dedent
from langchain.tools import tool
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from crewai import Agent, Task
from crewai import Crew
from crewai.crews.crew_output import CrewOutput

In [100]:
# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_PALM_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Configure the API
palm.configure(api_key= GOOGLE_API_KEY)
OPENAI_MODEL = "gpt-3.5-turbo"

In [101]:
def generate_text(prompt):
    completion = palm.generate_text(
        model="models/text-bison-001",
        prompt=prompt,
        temperature=0.7,
        max_output_tokens=800,
    )
    return completion.result

In [102]:
def parse_response(response):
    # Use regular expressions to find the sections more flexibly
    caviar_selection_match = re.search(r'Caviar Selection:\s*(.*)', response)
    champagne_pairing_match = re.search(r'Champagne Pairing:\s*(.*)', response)
    exclusive_insights_match = re.search(r'Exclusive Insights:\s*([\s\S]*)', response)

    caviar_selection = caviar_selection_match.group(1) if caviar_selection_match else None
    champagne_pairing = champagne_pairing_match.group(1) if champagne_pairing_match else None

    exclusive_insights = []
    if exclusive_insights_match:
        insights_section = exclusive_insights_match.group(1).strip()
        exclusive_insights = [line.strip().replace('*', '').strip() for line in insights_section.splitlines() if line.strip().startswith('*')]

    return caviar_selection, champagne_pairing, exclusive_insights

def caviar_coach(taste_profile, texture_preference, budget):
    image_url = "https://s.yimg.com/ny/api/res/1.2/.11X0wngxtZy7WtNp0r.2w--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyMDA7aD02NzQ-/https://media.zenfs.com/en/tasting_table_543/13b649b5330e192063fa2d41441150f3"
    prompt = f"""
    As a luxury caviar expert, provide a personalized recommendation based on the following preferences:
    Taste Profile: {taste_profile}
    Texture Preference: {texture_preference}
    Budget: ${budget}

    Please provide:
    1. A specific caviar recommendation
    2. A champagne pairing suggestion
    3. Some exclusive insights or tips for enjoying this caviar experience

    Format your response as follows:
    Caviar Selection: [Your recommendation]
    Champagne Pairing: [Your pairing suggestion]
    Exclusive Insights: [Your insights or tips]
    """

    try:
        # Get the generated response from the model
        response = generate_text(prompt)
        
        # Parse the response into sections using regular expressions
        caviar_selection, champagne_pairing, exclusive_insights = parse_response(response)
        
        # Format the response properly
        formatted_response = f"<div style='font-family: Arial, sans-serif; line-height: 1.6;'>"
        formatted_response += "<strong>Your Imperial Highness,</strong><br><br>"
        formatted_response += "Based on your exquisite preferences, I am honored to present our recommendation:<br><br>"
        caviar_selection.replace("**","")
        
        if caviar_selection:
            formatted_response += f"Caviar Selection: {caviar_selection}<br>"
        else:
            formatted_response += "<strong>Caviar Selection:</strong> Not available<br>"
        
        if champagne_pairing:
            formatted_response += f"Champagne Pairing: {champagne_pairing}<br><br>"
        else:
            formatted_response += "<strong>Champagne Pairing:</strong> Not available<br><br>"
        
        if exclusive_insights:
            formatted_response += "<strong>Exclusive Insights:</strong><br>"
            insights_html = "<br>".join([f"• {insight}" for insight in exclusive_insights if insight])  # Remove empty bullet points
            formatted_response += f"{insights_html}<br><br>"
        else:
            formatted_response += "<strong>Exclusive Insights:</strong> Not available<br><br>"
        # Add image to the response
        formatted_response += f"<img src='{image_url}' alt='Caviar Image' style='max-width: 50%; height: auto;'><br><br>"
        formatted_response += "May this experience elevate your senses and leave an indelible mark on your palate.<br><br>"
        formatted_response += "Bon appétit, Your Highness.<br><br>"
        formatted_response += "For more exclusive insights, visit us at: <a href='https://www.caviart.club' target='_blank'>here</a></div>"
        youtube_video_html = """<iframe width="560" height="315" src="https://www.youtube.com/embed/0QrUq7XWfvw" 
                             title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; 
                             encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>"""
        return formatted_response,youtube_video_html
    except Exception as e:
        return f"An error occurred: {str(e)}. Please check if the API key is set correctly."


In [132]:
class SearchTools():
  @tool("Search the internet")
  def search_internet(query):
    """Useful to search the internet
    about a a given topic and return relevant results"""
    top_result_to_return = 4
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': SERPER_API_KEY,
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    # check if there is an organic key
    if 'organic' not in response.json():
      return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
    else:
      results = response.json()['organic']
      string = []
      for result in results[:top_result_to_return]:
        try:
          string.append('\n'.join([
              f"Title: {result['title']}", f"Link: {result['link']}",
              f"Snippet: {result['snippet']}", "\n-----------------"
          ]))
        except KeyError:
          next

      return '\n'.join(string)

In [174]:
class RestaurantAgent():

  def __init__(self):
    self.OpenAIGPT35 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.7)

  def restaurant_expert(self):
    return Agent(
        role='Caviar Expert',
        goal='Find the  restaurants and bars that have caviar',
        backstory="""A culinary expert specializing in locating establishments with caviar.
        Resarch news, articles and food critics, bloggers, or industry experts directly for their recommendations on the best restaurants and bars that serve caviar
        """,
        tools=[
            SearchTools.search_internet
        ],
        verbose=True,
        llm = self.OpenAIGPT35
        )

In [176]:
class RestaurantsTasks():

    def find_restaurants_task(self, agent, city):
        return Task(description=dedent(f"""
            Retutn the list of restaurants or bars that have caviar in {city} or arround it.
            Resarch local news, advertidements, food critics, bloggers, or industry experts directly for their recommendations on the restaurants and bars that serve caviar
            """),
            agent=agent,
            expected_output="A list of restaurants and bars that have caviar including addresses"
        )

In [177]:
class RestaurantCrew:

    def __init__(self, city):
        self.city = city
    def run(self):
        agents = RestaurantAgent()
        tasks = RestaurantsTasks()

        # Initialize agents
        local_expert_agent = agents.restaurant_expert()
        try:
            # Create tasks for caviar restaurant/bar identification and gathering
            restaurants_finder_task = tasks.find_restaurants_task(
                local_expert_agent,
                self.city,
       
            )
            crew = Crew(
                agents=[ local_expert_agent ],
               tasks=[restaurants_finder_task],
               verbose=True
            )

            result = crew.kickoff()
            return result
        except Exception as e:
            print(f"Error occurred: {e}")
            raise

In [178]:
def on_click(city):
    restaurant_crew = RestaurantCrew(city)
    try:
        result = restaurant_crew.run()
        if isinstance(result, CrewOutput):  # Checking if the result is a CrewOutput object
            output_text = str(result)  # Convert CrewOutput object to string directly if it is already formatted
        else:
            output_text = "Unexpected result format."
        
        return output_text
    except Exception as e:
        print(f"An error occurred while generating the restaurant list: {e}")
        return "Something went wrong..."

In [179]:
def clear_recommendation():
    return ""

In [180]:
# Gradio interface for caviar coach function and restaurant finder
with gr.Blocks(title="🍾 Imperial Caviar Connoisseur 🥂") as demo:

    # Section for the caviar coach functionality
    with gr.Column():
        # User preferences for caviar coach
        preferred_taste = gr.Dropdown(["Mild & Delicate", "Rich & Buttery", "Bold & Intense"], label="Preferred Taste Profile")
        preferred_texture = gr.Dropdown(["Soft & Creamy", "Firm & Distinct"], label="Preferred Texture")
        budget = gr.Slider(minimum=50, maximum=1000, step=50, label="Budget (USD)")
        
        # Recommendation output
        recommendation_output = gr.HTML(label="Your Royal Recommendation")

        # Buttons in a row below the budget slider
        with gr.Row():
            submit_button = gr.Button("Get Caviar Recommendation")
            clear_button = gr.Button("Clear Results")

    # When submit_button is clicked, trigger caviar_coach
    submit_button.click(
        fn=caviar_coach,
        inputs=[preferred_taste, preferred_texture, budget], # Use a hidden textbox for city
        outputs=[recommendation_output]
    )

    # When clear_button is clicked, clear the recommendation output
    clear_button.click(
        fn=clear_recommendation,
        inputs=[],
        outputs=[recommendation_output]
    )

    # Section for finding caviar restaurants
    with gr.Column():
        # City input for finding restaurants
        city_input = gr.Textbox(label="Enter your city to find caviar places")
         # Buttons in a row below the budget slider
        with gr.Row():
            find_caviar_button = gr.Button("Find Caviar Places")
            clear_places_button = gr.Button("Clear Results")

        # Restaurant search output
        restaurant_output = gr.Textbox(label="Here are the places you can enjoy caviar")

    # When find_caviar_button is clicked, trigger on_click for restaurant search
    find_caviar_button.click(
        fn=on_click,
        inputs=[city_input],
        outputs=[restaurant_output]
    )

    clear_places_button.click(
        fn=clear_recommendation,
        inputs=[],
        outputs=[restaurant_output]
    )

In [181]:
if __name__ == "__main__":
   demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7892
Running on public URL: https://a14c693b28001a1907.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


2024-08-23 17:08:58,298 - 42232 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-23 17:08:58][DEBUG]: == Working Agent: Caviar Expert
 [2024-08-23 17:08:58][INFO]: == Starting Task: 
Retutn the list of restaurants or bars that have caviar in New York or arround it.
Resarch local news, advertidements, food critics, bloggers, or industry experts directly for their recommendations on the restaurants and bars that serve caviar



> Entering new CrewAgentExecutor chain...
I need to search the internet for recommendations on restaurants and bars that serve caviar in New York.

Action: Search the internet
Action Input: {'query': {'title': 'best restaurants and bars with caviar in New York'}} 

Title: jquery - What does [object Object] mean? (JavaScript) - Stack Overflow
Link: https://stackoverflow.com/questions/8892465/what-does-object-object-mean-javascript
Snippet: It means you are alerting an instance of an object. When alert ing the object, toString() is called on the object, and the default ...

-----------------
Title: [object, object] in JavaScript – Mean